- For those who are not familiar with pytorch, <br>
    try `pytorch_tutorial.ipynb` in advance of HW4_2

# Neural Network (pytorch) : CIFAR-100

- CIFAR-100 has 100 classes containing 600 32x32 colour images each. 
- The 100 classes in the CIFAR-100 are grouped into 20 superclasses.
- There are 500 training images and 100 testing images per class.

In [ ]:
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn

from utils import *
from YourAnswer import simple_CNN, deep_CNN

## Random Seed
- Fix the seed to constraint the randomness and reproduce the results
- np.random, CUDNN, CUDA seed

In [ ]:
seed = 0
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

## Data Preprocessing

- Load the CIFAR-100 dataset
- data is loaded by the size of batch

In [ ]:
# data loader
batch_size = 32
trainloader, testloader, classes = dataloader('CIFAR100', batch_size)
num_classes = len(classes)

## Visualize training images
- Check what the data looks like

In [ ]:
sample = iter(trainloader)
sample_image, sample_label = sample.next()

imshow(torchvision.utils.make_grid(sample_image,8))

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

### To do:
- Implement `simple_CNN` in `YourAnswer.py` file <br> 
- Feature extraction is given
- You need to fill the Classifier

In [ ]:
net = simple_CNN(num_classes)
net.to(device)

## Set the loss and optimizer
loss : Cross Entropy Loss <br>
optimizer : Stochastic Gradient Descent

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)

## Initialize the loss and accuracy

In [ ]:
epochs = 20

global trn_loss 
global tst_loss 
global trn_acc
global tst_acc

trn_loss = torch.zeros(epochs)
tst_loss= torch.zeros(epochs)
trn_acc = torch.zeros(epochs)
tst_acc = torch.zeros(epochs)

## Train 
- Send the images and labels to the devices for the computation using GPU
- Need to empty the gradients using the zero_grad() at first
- Compute the loss and update the weights using the optimizer


In [ ]:
def train(epoch):
    print('\nEpoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (images, labels) in enumerate(trainloader):

        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = net(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

        progress_bar(batch_idx, len(trainloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)' % (train_loss/(batch_idx+1), 100.*correct/total, correct, total))

    trn_loss[epoch] = train_loss/(batch_idx+1)
    trn_acc[epoch] = 100.*correct/total

## Test
- Model should be set to eval() mode
- Model should not be updated during the test : no_grad() is needed
- Best accuracy and model are selected according to the test accuracy

In [ ]:
def test(epoch):
    global best_acc
    global best_net
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (images, labels) in enumerate(testloader):


            images = images.to(device)
            labels = labels.to(device)

            outputs = net(images)
            
            loss = criterion(outputs, labels)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

            progress_bar(batch_idx, len(testloader), 'Test Loss: %.3f | Test Acc: %.3f%% (%d/%d)' % (test_loss/(batch_idx+1), 100.*correct/total, correct, total))

    acc = 100.*correct/total
    if acc > best_acc:
        best_acc = acc
        best_net = net
        
    tst_loss[epoch] = test_loss/(batch_idx+1)
    tst_acc[epoch] = acc


- Machine starts learning

In [ ]:
start = time.time()
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 

for epoch in range(start_epoch, start_epoch+epochs):
    train(epoch)
    test(epoch)

#Time Calculation
finish = time.time() - start
temp_log = "\nTime Elapse: %s" %(format_time(finish))
print(temp_log)


In [ ]:
loss_and_acc(trn_acc, trn_loss, 'Train', epochs)

In [ ]:
loss_and_acc(tst_acc, tst_loss, 'Test', epochs)

In [ ]:
print('The best test accuracy : %d' %best_acc, '% at epoch', torch.argmax(tst_acc).item())

In [ ]:
print('Saving the best model..')
savePath = "./CIFAR100_simple.pth" 
torch.save(best_net.state_dict(), savePath)

### To do:
- Implement `deep_CNN` in `YourAnswer.py` file <br> 

- You have two choices. __<br>1. Build any model as you wish <br>2. Follow the instruction in `YourAnswer.py` file__

- Whatever your choice, score would be the same as long as the model achieves more than $\Large60\%$  test accuracy  

- The epoch should be no more than $\Large40 $ (You can change the epoch if you want, but the max epoch is 40)

- Do __NOT__ change the code below : <br>
    net = deep_CNN(num_classes)

In [ ]:
net = deep_CNN(num_classes)
net.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)

epochs = 40

global trn_loss 
global tst_loss 
global trn_acc
global tst_acc

trn_loss = torch.zeros(epochs)
tst_loss= torch.zeros(epochs)
trn_acc = torch.zeros(epochs)
tst_acc = torch.zeros(epochs)


In [ ]:
start = time.time()
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch

for epoch in range(start_epoch, start_epoch+epochs):
    train(epoch)
    test(epoch)

#Time Calculation
finish = time.time() - start
temp_log = "\nTime Elapse: %s" %(format_time(finish))
print(temp_log)


In [ ]:
loss_and_acc(trn_acc, trn_loss, 'Train', epochs)

In [ ]:
loss_and_acc(tst_acc, tst_loss, 'Test',epochs)

In [ ]:
print('The best test accuracy : %d' %best_acc, '% at epoch', torch.argmax(tst_acc).item())

In [ ]:
print('Saving the best model..')
savePath = "./CIFAR100_deep.pth" 
torch.save(best_net.state_dict(), savePath)